In [1]:
import logfire

logfire.configure()

Logfire project URL: ]8;id=418909;https://logfire.pydantic.dev/jared-neumann/ensi\https://logfire.pydantic.dev/jared-neumann/ensi]8;;\


In [2]:
example_dreams = []

import os

# Load the example dreams from directory
for file in os.listdir('../data/dreams_and_interpretations'):
    if file.endswith('.txt'):
        with open(os.path.join('../data/dreams_and_interpretations', file), 'r') as f:
            example_dreams.append(f.read())

print(example_dreams)

['Death-Dream of Enkidu (Thompson Epic, pl. 29 IV: 14-54)\n\n"Dear friend, this night I had dreams:\n\nThe heaven cried, the earth echoed. I stood alone! There came a griffin(?) with a gloomy mien, his face was like that of a lion. His talons were those of an eagle. He summoned me with strong words, his hoofs stomped the ground. He stripped me of my garments.\n\n(In the dream) he submerged me.\n\n(eight lines destroyed)\n\nHe turned me into a double of his body. My arms were now clad with feathers like those of a bird. Holding me by the hand, he led me down to the Dark Mansion, the abode of Irkalla. To the house which he who enters will not leave anymore, upon a road on which there is no way back. To the house whose inhabitants are bereft of light, where earth is their fare and clay their food. Clad like birds in feather-garments, they sit in darkness, never to see light again.\n\nTo the House of Dust, where I entered, I beheld all the kings, their mitres put away. I saw the princes, a

In [3]:
# put the examples into a dataframe
import pandas as pd

df = pd.DataFrame(example_dreams, columns=['dream'])

print(df)


                                                dream
0   Death-Dream of Enkidu (Thompson Epic, pl. 29 I...
1   Dream of Nabonidus (V R 64 1: 13-55)\n\nDuring...
2   A Dream of The King (KUB XV 5 III: 4-14)\n\n"A...
3   Another Dream of Hattushili (op. cit., p. 46, ...
4   Dream of Gudea (Gudea, Cyl. A IV: 7-VI: 14 = T...
5   The Crossing of the River Idid'e (Ashurbanipal...
6   Dream of the Prince of Bekhten (Bentresh Stela...
7   Dream of Thutmose IV (Sphinx Stela, lines 8-12...
8   First Dream-Appearance of Ishtar (MVAeG 29(3):...
9   Dreams of Gilgamesh (Neo-Assyrian version, Tho...
10  Dream of Taimhotep (Stela, lines 1-15)\n\nA bo...
11  Tages, the Twin-Sister (From no. 77 Col. I-II)...
12  A Warning Dream (Ashurbanipal, V R 2, III: 118...
13  Dreams from a Religious Poem (Ludlul bel nemeq...
14  Ptolemaios to Damoxenos (From no. 78)\n\nPtole...
15  Dream of Ptolemy Soter (Plutarch, De Iside 28;...
16  Dream of Enkidu (Hittite, KUB VIII 48 I: 2-22,...
17  Dream of Tanutamun (Stel

In [28]:
# add a column for prospective symbols
df['omens'] = None

print(df)


                                                dream  \
0   Death-Dream of Enkidu (Thompson Epic, pl. 29 I...   
1   Dream of Nabonidus (V R 64 1: 13-55)\n\nDuring...   
2   A Dream of The King (KUB XV 5 III: 4-14)\n\n"A...   
3   Another Dream of Hattushili (op. cit., p. 46, ...   
4   Dream of Gudea (Gudea, Cyl. A IV: 7-VI: 14 = T...   
5   The Crossing of the River Idid'e (Ashurbanipal...   
6   Dream of the Prince of Bekhten (Bentresh Stela...   
7   Dream of Thutmose IV (Sphinx Stela, lines 8-12...   
8   First Dream-Appearance of Ishtar (MVAeG 29(3):...   
9   Dreams of Gilgamesh (Neo-Assyrian version, Tho...   
10  Dream of Taimhotep (Stela, lines 1-15)\n\nA bo...   
11  Tages, the Twin-Sister (From no. 77 Col. I-II)...   
12  A Warning Dream (Ashurbanipal, V R 2, III: 118...   
13  Dreams from a Religious Poem (Ludlul bel nemeq...   
14  Ptolemaios to Damoxenos (From no. 78)\n\nPtole...   
15  Dream of Ptolemy Soter (Plutarch, De Iside 28;...   
16  Dream of Enkidu (Hittite, K

In [21]:
# load the modern rules
rules = pd.read_json('../data/rules_for_interpretation/modernized_rules.json', orient='records', lines=True)

rules

,rule,omen,interpretation
0,"If a person dreams of turning into a lion, the...",turning into a lion,they will experience significant losses
1,"If a person dreams of interacting with a bull,...",interacting with a bull,their household will become prosperous
2,If a person dreams of having dealings with a b...,having dealings with a bull,they will gain whatever the gods bestow upon ...
3,If a person dreams of holding a sprout in thei...,holding a sprout in their lap and kissing it r...,they will gain wealth in the form of barley a...
4,If a person dreams of a sprout growing out of ...,a sprout growing out of their lap,they will lose all that they own
...,...,...,...
346,If a person dreams of carrying beer-wort in th...,carrying beerwort in the street,grief is in store for them
347,If a person dreams of carrying water in the st...,carrying water in the street,their misfortune will depart
348,If a person dreams of carrying a table in the ...,carrying a table in the street,they will die through the wrath caused by the...
349,If a person dreams of a god uttering a benedic...,a god uttering a benediction before them,they will experience the wrath of the god


In [30]:
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.models.ollama import OllamaModel
from pydantic_ai.exceptions import UnexpectedModelBehavior
import random
import asyncio
from pydantic import Field
from typing import List, Optional

ollama_model = OllamaModel(
    model_name='llama3.2:3b',
    base_url='http://localhost:11434/v1',
)

class Omens(BaseModel):
    symbolic_sentences: List[str] = Field(
        default=[],
        description="A list of simple sentences describing a part of a dream that might be an omen in need of interpretation. They should be in the form of someone doing something or having something happen to them. They should be in the form of a subject, verb, and object.",
        examples=[
            "a person has lame feet",
            "a person is given a door",
            "a person goes to geneva",
        ]
    )

agent = Agent(
    model=ollama_model,
    result_type=Omens,
    retries=16,
    result_retries=16
)

async def get_omens(agent, dream):  # Change this line to make it async

    try:
        result = await agent.run(
            dream
        )
    except Exception as e:
        print(e)
        return []
    
    logfire.info(f"Dream: {dream}")
    logfire.info(f"Omens: {result.data}")

    return result.data

async def main():
    df['omens'] = await asyncio.gather(*[get_omens(agent, x) for x in df['dream']])

await main()

20:22:12.068 agent run prompt=Death-Dream of Enkidu (Thompson Epic, pl. 29 IV: 14-54)

"Dear...nd saw me, saying: 'Who has brought this man hither?'

(break)
20:22:12.069   preparing model and tools run_step=1
20:22:12.070   model request
20:22:12.073 agent run prompt=Dream of Nabonidus (V R 64 1: 13-55)

During my lawful rule, t...ing of the Great Lord Marduk, whose command cannot be changed.
20:22:12.075   preparing model and tools run_step=1
20:22:12.075   model request
20:22:12.079 agent run prompt=A Dream of The King (KUB XV 5 III: 4-14)

"A dream of His Maje...-instruments and the lapis lazuli stones to the great deity.'"
20:22:12.080   preparing model and tools run_step=1
20:22:12.080   model request
20:22:12.084 agent run prompt=Another Dream of Hattushili (op. cit., p. 46, KBo VI 29 1: 16-...d of the goddess. The goddess entrusted her to me in a dream."
20:22:12.084   preparing model and tools run_step=1
20:22:12.085   model request
20:22:12.088 agent run prompt=Dream of Gudea

In [8]:
print(result._cost)

Cost(request_tokens=175, response_tokens=24, total_tokens=199, details=None)
